# Triangular Arbitrage (toy)
Detect cycles >1 on synthetic BTC/ETH/USDT.

In [ ]:
import numpy as np
btcusdt=68000.0; ethusdt=3700.0; btceth=btcusdt/ethusdt
cycle=btcusdt/(ethusdt*btceth)
print('cycle',cycle,'>1 is opp? ->',cycle>1.0001)